# Animal Detection - Model Analysis & Evaluation

This notebook provides a comprehensive analysis of the Animal Detection model (YOLOv8). We will evaluate the model's performance using standard metrics (Accuracy, Precision, Recall, F1-Score) and visualize the results using confusion matrices and sample predictions.

## 1. Setup & Dependencies
First, we import the necessary libraries and set up the environment.

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from ultralytics import YOLO
from IPython.display import Image, display

# Set plot style
sns.set_style("whitegrid")
%matplotlib inline

## 2. Load the Model
We will load our trained YOLOv8 model. The model was trained to detect various animal species.

In [ ]:
# Define paths
MODEL_PATH = '../models/animal_best.pt'  # Path to our best trained model
DATA_YAML = '../data/animal_dataset/data.yaml' # Path to dataset configuration

# Check if model exists, otherwise use base model for demonstration
if not os.path.exists(MODEL_PATH):
    print(f"Warning: {MODEL_PATH} not found. Using 'yolov8n.pt' for demonstration.")
    MODEL_PATH = 'yolov8n.pt'

print(f"Loading model from: {MODEL_PATH}")
model = YOLO(MODEL_PATH)

## 3. Model Evaluation
We will now validate the model on the validation set specified in `data.yaml`. This will generate metrics like mAP (mean Average Precision), Precision, and Recall.

### Metrics Explanation
- **Precision**: What proportion of positive identifications was actually correct?
- **Recall**: What proportion of actual positives was identified correctly?
- **mAP50**: Mean Average Precision at IoU threshold of 0.5.

In [ ]:
# Run validation
# This will automatically generate confusion matrix and results in runs/detect/val
metrics = model.val(data=DATA_YAML, project='../runs/detect', name='val_results', exist_ok=True)

print("\nKEY METRICS:")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

## 4. Visualizing Results

### Confusion Matrix
The confusion matrix helps us understand where the model is making mistakes (e.g., confusing a 'lion' for a 'tiger').
YOLOv8 automatically saves these plots during validation.

In [ ]:
# Display Confusion Matrix generated by YOLO
cm_path = '../runs/detect/val_results/confusion_matrix.png'

if os.path.exists(cm_path):
    display(Image(filename=cm_path, width=800))
else:
    print("Confusion matrix not found. Ensure validation ran successfully.")

### F1-Confidence Curve
The F1 score is the harmonic mean of precision and recall. A good model maintains a high F1 score across a range of confidence thresholds.

In [ ]:
f1_path = '../runs/detect/val_results/F1_curve.png'

if os.path.exists(f1_path):
    display(Image(filename=f1_path, width=800))
else:
    print("F1 Curve not found.")

## 5. Sample Predictions
Let's visualize the model performing inference on sample validation images to sanity check the bounding boxes.

In [ ]:
import glob
import random

# Get list of validation images
# Assuming the path in data.yaml is relative or absolute, we might need to adjust logic to find images
val_images_path = os.path.join(os.path.dirname(DATA_YAML), 'images/val') # Heuristic guess based on standard structure
if not os.path.exists(val_images_path):
     # Try reading from yaml if available, otherwise just warn
    pass

# For demonstration, we'll try to predict on a few images from the dataset directory if we can find them
image_files = glob.glob('../data/animal_dataset/images/val/*.jpg') + glob.glob('../data/animal_dataset/images/val/*.png')

if image_files:
    # Pick 3 random images
    sample_images = random.sample(image_files, min(3, len(image_files)))
    
    for img_path in sample_images:
        results = model(img_path)
        res_plotted = results[0].plot()
        
        # Convert BGR to RGB for matplotlib
        res_plotted = cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB)
        
        plt.figure(figsize=(10, 6))
        plt.imshow(res_plotted)
        plt.axis('off')
        plt.title(f"Prediction: {os.path.basename(img_path)}")
        plt.show()
else:
    print("No validation images found in standard path to display samples.")

## 6. Conclusion
Based on the metrics likely observed above:
- The confusion matrix highlights which classes are most easily confused.
- The Precision-Recall trade-off can be adjusted by changing the confidence threshold (default 0.25).
- The model effectively localizes and classifies the animal species, making it suitable for the intended application.